We have already extracted $H^{FMR}$ and $\Delta H$ from FMR spectra for each excitation microwave frequency. That is, from the set {$f_i$, $H_{ij}$, $dP/dH_{ij}$} we have extracted {$f_i$, $H^{FMR}_i$, $\Delta H_i$} as well as the covariances $$V_{ij} = \begin{bmatrix} \sigma_{H^{FMR}_i}^2 & Cov\left(H^{FMR}_i, \Delta H_i\right) \\ Cov\left(H^{FMR}_i, \Delta H_i\right) & \sigma_{\Delta H_i}\end{bmatrix}$$
We know that $$f = \frac{g\mu_b\mu_0}{h}\left[H_{FMR}-M_{eff}\right]$$ and $$\Delta H = \Delta H_0 + \frac{h}{g\mu_B\mu_0} \alpha f$$ but since the two variables are correlated, we will use $f$ as the "$x$" value and fit the "$y$" values together using a linear least squares fit $$\begin{align} \vec{\mu} &= A\vec{\theta} \\ \begin{bmatrix}H_{FMR} \\ \Delta H\end{bmatrix} &= \begin{bmatrix}1 & f & 0 & 0 \\ 0 & 0 & 1 & f\end{bmatrix} \begin{bmatrix} M_{eff} \\ \frac{h}{g\mu_B\mu_0} \\ \Delta H_0 \\ \frac{h\alpha}{g\mu_B\mu_0} \end{bmatrix}\end{align}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob,os

In [ ]:
#extract parameters from files
infile = '../ppblsmo20A-OOP-redo-highsens/ppblsmo20A-20190221-peakfits.npz'
data = np.load(infile)

In [ ]:
data.files

In [ ]:
sort = np.argsort(data['frequenciesGHz'])
freqs = data['frequenciesGHz'][sort]
fitparams = data['LorentzianFitParamsSeparated'][sort]
fitcov = data['LorentzianFitCovariancesSeparated'][sort]
fitcovfull = data['LorentzianFitCovarianceFull'][sort]

In [ ]:
mainpeak = []
for i,f in enumerate(fitparams):
    NPeaks = len(f)
    peakamplitudes = [a[0] for a in f]
    mainpeak.append(np.argmax(peakamplitudes)) #the main peak is the one with the largest amplitude
    #maybe it should be the one with the lowest chi^2 with only that peak?
    
print mainpeak

In [ ]:
Y = np.zeros(2*len(freqs))
A = np.zeros((2*len(freqs), 4))
V = np.zeros((2*len(freqs), 2*len(freqs)))

for i,f in enumerate(freqs):
    params = [1,3] #H_FMR, \Delta H
    
    Y[2*i] = fitparams[i][mainpeak[i]][params[0]] #H_FMR
    Y[2*i + 1] = fitparams[i][mainpeak[i]][params[1]] #Delta\H
    
    A[2*i][0] = 1.
    A[2*i][1] = f
    A[2*i + 1][2] = 1.
    A[2*i + 1][3] = f
    
    V[2*i:2*i+2, 2*i:2*i+2] = fitcov[i][mainpeak[i]][params][:,params]

In [ ]:
#exact calculation instead of minimizing chi^2 (which could lead to some errors if the minimum is very shallow)
Vinv = np.linalg.inv(V)
AtVinv = np.matmul(A.transpose(),Vinv)
varTheta = np.linalg.inv(np.matmul(AtVinv,A))
theta = np.matmul(np.matmul(varTheta,AtVinv),Y)

def chi2(Y, A, V, theta):
    Vinv = np.linalg.inv(V)
    diff = Y - np.matmul(A,theta)
    chi2 = np.matmul(np.matmul(diff.transpose(),Vinv),diff)
    return chi2

mu = np.matmul(A,theta)
chi2min = chi2(Y,A,V,theta)
df = len(Y) - len(theta)
print "goodness of fit: "+str(chi2min/df)

In [ ]:
%matplotlib inline
plt.errorbar(freqs,Y[::2],yerr=[np.sqrt(v[i]) for i,v in enumerate(V)][::2],fmt='.',label='data')
plt.plot(freqs,mu[::2],'-',label='fit')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Resonant Field (Oe)')
plt.legend()

In [ ]:
%matplotlib inline
plt.errorbar(freqs,Y[1::2],yerr=[np.sqrt(v[i]) for i,v in enumerate(V)][1::2],fmt='.',label='data')
plt.plot(freqs,mu[1::2],label='fit')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Linewidth (Oe)')
plt.legend()

In [ ]:
Meff = theta[0]/(4.*np.pi)
eMeff = np.sqrt(varTheta[0][0])/(4.*np.pi)
print 'Meff = '+str(Meff)+' pm '+str(eMeff)+' emu/cc'

In [ ]:
h = 6.62607004e-34*1e7 #erg-s
muB = 9.274000999e-21 #erg/G
mu0 = 1.00000037 #G/Oe - https://www.engineeringtoolbox.com/permeability-d_1923.html
giga = 1e9

gop = h/(muB*mu0)*giga/theta[1]
egop = np.sqrt(varTheta[1][1])*gop/theta[1]
print 'gop = '+str(gop)+' pm '+str(egop)

In [ ]:
deltaH0 = theta[2]
edeltaH0 = np.sqrt(varTheta[2][2])
print 'delta H 0 = '+str(deltaH0)+' pm '+str(edeltaH0)

In [ ]:
alpha = theta[3]/theta[1]
ealpha = np.sqrt(varTheta[3][3]/theta[1]**2 + varTheta[1][1]*theta[3]**2/theta[1]**4 -2.*varTheta[1][3]*theta[3]/theta[1]**3)
print 'alpha = '+str(alpha)+' pm '+str(ealpha)

In [ ]:
alphas = np.linspace(alpha-3.*ealpha,alpha+3.*ealpha,100)
chis = [chi2(Y, A, V, np.array(list(theta[:3])+[a*theta[1]])) for a in alphas]

In [ ]:
%matplotlib inline
import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
q = st.chi2.ppf(CLs,4)

plt.plot(alphas,chis)
for i,b in enumerate(q):
    plt.plot(alphas,np.ones_like(alphas)*(np.min(chis)+b),label=str(CLs[i]*100.)+'%')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\chi^2$')
plt.legend()

In [ ]:
gs = np.linspace(gop-3.*egop,gop+3.*egop,100)
chis = [chi2(Y, A, V, np.array([theta[0],h/(muB*mu0)*giga/g,theta[2],theta[3]])) for g in gs]
#chis = [[chi2(Y, A, V, np.array([theta[0],g/h*(muB*mu0)/giga,theta[2],a*g/h*(muB*mu0)/giga])) for g in gs] for a in alphas]

In [ ]:
%matplotlib inline
import scipy.stats as st
CLs = [0.68, 0.95, 0.99]
q = st.chi2.ppf(CLs,4)

plt.plot(gs,chis)
for i,b in enumerate(q):
    plt.plot(gs,np.ones_like(gs)*(np.min(chis)+b),label=str(CLs[i]*100.)+'%')
plt.xlabel(r'$g_{op}$')
plt.ylabel(r'$\chi^2$')
plt.legend()

In [ ]:
chis = [[chi2(Y, A, V, np.array([theta[0],h/(muB*mu0)*giga/g,theta[2],a*h/(muB*mu0)*giga/g])) for g in gs] for a in alphas]

In [ ]:
chis = np.array(chis)

ind = np.unravel_index(np.argmin(chis, axis=None), chis.shape)
print ind

m = np.min(chis)
levels = m+np.array(q)
    
%matplotlib inline

fig1, ax1 = plt.subplots()

# Basic contour plot
CS1 = ax1.contour(alphas,gs,chis,levels)
ax1.plot(alpha,gop,'o')
ax1.plot(alphas[ind[1]],gs[ind[0]],'*')

fmt = {}
for l, s in zip(CS1.levels, [str(c*100.)+'%' for c in CLs]):
    fmt[l] = s

# Label every other level using strings
ax1.clabel(CS1, CS1.levels, inline=True, fmt=fmt)#, fontsize=10)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$g_{op}$')
plt.title(r'$\chi^2$ Contour plot')

#s = allparams.shape
#for l in np.arange(0,s[0]):
#    for m in np.arange(0,s[1]):
        

#print allparams.shape
#model = FMR.LorentzianDerivativeNWrapper(Hs[i],allparams)
#making a triangle plot

covag = theta[3]*varTheta[1][1]*g/theta[1]**3 -g/theta[1]**2*varTheta[1][3]
print 'rho = '+str(covag/(ealpha*egop))